In [ ]:
pip install transformers torch nltk

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.1 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pdfplumber
import os

from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import csv

from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Loading BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set the model to evaluation mode
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def get_embedding(word):
    inputs = tokenizer(word, return_tensors='pt', truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device as the model
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move the output to CPU before converting to numpy
    return embedding

def ngrams(tokens, n):
    return zip(*[tokens[i:] for i in range(n)])

def tokenize_document(text, n_gram_range=(1, 3)):
    tokens = tokenizer.tokenize(text)
    tokens = [token for token in tokens if not token.startswith('##')]  # Remove subword tokens
    tokens = [token for token in tokens if token.lower() not in stop_words]  # Remove stopwords
    n_grams = list(chain.from_iterable(ngrams(tokens, n) for n in range(n_gram_range[0], n_gram_range[1] + 1)))
    unique_n_grams = list(set([' '.join(n_gram) for n_gram in n_grams]))  # Remove duplicates and join n-grams
    return unique_n_grams

def compute_document_embeddings(tokens):
    embeddings = {token: get_embedding(token) for token in tokens}
    return embeddings

# Predefined set of words
predefined_words_v0 = ["learning task","classification", "regression","clustering","meta knowledge", "surrogate models"
                    "meta model","base learner", "meta knowledge database", "meta features", "task properties",
                    "meta learning", "metalearning", "advisory method"]

predefined_words = ["learning task","classification", "regression","clustering","meta knowledge", "surrogate models",
                    "meta model","base learner", "meta knowledge database", "meta features", "task properties",
                    "meta learning", "metalearning",  "advisory method", "domain knowledge", "deep learning model", "deep meta learning",
                    "deep learning method", "classification model", "candidate classification", "base learning", "recommendation model",
                    "meta learning task", "meta learning database", "meta based learning", "feature learning", "ensemble classification model"]

# Compute embeddings for predefined words
predefined_embeddings = {word: get_embedding(word) for word in predefined_words}

In [ ]:
def find_similar_words_in_document(predefined_embeddings, document_embeddings, min_similarity=0.97):
    similar_words = []

    for predefined_word, predefined_embedding in predefined_embeddings.items():
        for doc_word, doc_embedding in document_embeddings.items():
            similarity = cosine_similarity(predefined_embedding, doc_embedding)[0][0]
            if similarity > min_similarity:
                similar_words.append((doc_word, similarity))

    return similar_words

In [ ]:
# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^A-Za-z\s]', '', text)
    return text

def lemmatize_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    return ' '.join(words)

def read_pdf_content(pdf_path):
    content = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text(x_tolerance=2, y_tolerance=2)
            if text:
                cleaned_text = clean_text(text)
             #   lemmatized_text = lemmatize_text(cleaned_text)
                content.append(cleaned_text)

    return "\n\n".join(content)

In [ ]:
# Specify the path to your PDF file
folder_path = "./pdfs/"
for filename in os.listdir(folder_path):
  print(filename)
  pdf_path = os.path.join(folder_path, filename)
  pdf_content = read_pdf_content(pdf_path)
  # Tokenize and compute embeddings for the document
  document_tokens = tokenize_document(pdf_content)
  document_embeddings = compute_document_embeddings(document_tokens)
  # Find similar words in the document
  similar_words = find_similar_words_in_document(predefined_embeddings, document_embeddings)

  # Write similar words and similarity scores to a CSV file
  output_file = os.path.join(folder_path, f"{os.path.splitext(filename)[0]}_similar_words.csv")
  with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Word", "Similarity"])
    for word, score in similar_words:
        writer.writerow([word, f"{score:.4f}"])

liu2023federated.pdf
hattie1996effects.pdf
langdon2022meta.pdf
